In [1]:
import pandas as pd
import plotly.express as px
from script_weather import weather_score_df

In [34]:
df_final = weather_score_df()

File not found for department 2A in 1950-2023
File not found for department 2A in 2024-2025
File not found for department 2B in 1950-2023
File not found for department 2B in 2024-2025


In [3]:
df_final = pd.read_csv('data_risk_weather.csv')

In [5]:
df_final['code_commune_INSEE'] = df_final['code_commune_INSEE'].astype(str).str.zfill(5)
df_final['department'] = df_final['department'].astype(str).str.zfill(2)

In [6]:
df_final.head()

,code_commune_INSEE,year,drought_score,heat_score,rainfall_score,extrem_events_score,department
0,01014,2014,1,1,2,1,01
1,01014,2015,2,2,2,1,01
2,01014,2016,1,1,2,1,01
3,01014,2017,1,1,2,1,01
4,01014,2018,1,1,2,1,01


In [31]:
df_final[df_final.isna().any(axis=1)]

,code_commune_INSEE,year,drought_score,heat_score,rainfall_score,extrem_events_score
1197,04158,2021,0.0,0.0,NaN,0
3055,09032,2021,0.0,0.0,NaN,0
21900,66188,2022,NaN,NaN,0.0,1


In [35]:
df_final.fillna(0, inplace=True)

In [36]:
df_final.isna().sum()

code_commune_INSEE     0
year                   0
drought_score          0
heat_score             0
rainfall_score         0
extrem_events_score    0
dtype: int64

In [37]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29878 entries, 0 to 29877
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   code_commune_INSEE   29878 non-null  object 
 1   year                 29878 non-null  int32  
 2   drought_score        29878 non-null  float64
 3   heat_score           29878 non-null  float64
 4   rainfall_score       29878 non-null  float64
 5   extrem_events_score  29878 non-null  int64  
dtypes: float64(3), int32(1), int64(1), object(1)
memory usage: 1.3+ MB


In [38]:
columns = ['drought_score', 'heat_score', 'rainfall_score']

for column in columns:
    df_final[column] = df_final[column].astype(int)

In [39]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29878 entries, 0 to 29877
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   code_commune_INSEE   29878 non-null  object
 1   year                 29878 non-null  int32 
 2   drought_score        29878 non-null  int64 
 3   heat_score           29878 non-null  int64 
 4   rainfall_score       29878 non-null  int64 
 5   extrem_events_score  29878 non-null  int64 
dtypes: int32(1), int64(4), object(1)
memory usage: 1.3+ MB


In [5]:
df_final['year'].unique()

array([2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024],
      dtype=int32)

In [40]:
df_final['department'] = df_final['code_commune_INSEE'].astype(str).str[:2]
df_final.head()

,code_commune_INSEE,year,drought_score,heat_score,rainfall_score,extrem_events_score,department
0,01014,2014,1,1,2,1,01
1,01014,2015,2,2,2,1,01
2,01014,2016,1,1,2,1,01
3,01014,2017,1,1,2,1,01
4,01014,2018,1,1,2,1,01


In [13]:
df_final['department'].nunique()

94

In [12]:
df_final['code_commune_INSEE'].nunique()

3500

In [42]:
df_final.to_csv('data_risk_weather.csv', index=False)

In [6]:
df_final.shape

(29878, 6)

In [6]:
df_final['code_departement'].nunique()

94

In [11]:
# df_grouped = df_final.groupby(['departement', 'year'])['avg_risk_score'].mean().reset_index()

px.density_heatmap(
    df_final,
    x='year',
    y='department',
    z='drought_score',
    color_continuous_scale='OrRd',
    title='Score moyen de risque climatique par département et trimestre'
)


In [7]:
import geopandas as gpd

gdf = gpd.read_file(
    "https://raw.githubusercontent.com/gregoiredavid/france-geojson/master/departements.geojson"
)

print(gdf.columns)

Index(['code', 'nom', 'geometry'], dtype='object')


In [9]:
gdf_merged = gdf.merge(df_final, 
                       left_on='code', 
                       right_on='department', 
                       how='inner') \
                .drop(columns='department')
gdf_merged.head()

,code,nom,geometry,code_commune_INSEE,year,drought_score,heat_score,rainfall_score,extrem_events_score
0,01,Ain,"POLYGON ((4.78021 46.17668, 4.78024 46.18905, ...",01014,2014,1,1,2,1
1,01,Ain,"POLYGON ((4.78021 46.17668, 4.78024 46.18905, ...",01014,2015,2,2,2,1
2,01,Ain,"POLYGON ((4.78021 46.17668, 4.78024 46.18905, ...",01014,2016,1,1,2,1
3,01,Ain,"POLYGON ((4.78021 46.17668, 4.78024 46.18905, ...",01014,2017,1,1,2,1
4,01,Ain,"POLYGON ((4.78021 46.17668, 4.78024 46.18905, ...",01014,2018,1,1,2,1


In [ ]:
fig = px.choropleth(
    gdf_merged,
    geojson=gdf,
    locations='code',            # colonne dans ton DataFrame
    featureidkey='properties.code',    # chemin dans le GeoJSON
    color='drought_score',
    animation_frame='year',    # variable temporelle
    color_continuous_scale='Reds',
    range_color=(0, 3),                # 0 = Low, 3 = Critical
    labels={'drought_score': 'Drought Score'},
    title='Drought Score by Department Over Time'
)

fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig.show()

In [ ]:
fig = px.choropleth(
    gdf_merged,
    geojson=gdf,
    locations='code',            # colonne dans ton DataFrame
    featureidkey='properties.code',    # chemin dans le GeoJSON
    color='heat_score',
    animation_frame='year',    # variable temporelle
    color_continuous_scale='Reds',
    range_color=(0, 3),                # 0 = Low, 3 = Critical
    labels={'heat_score': 'heat Score'},
    title=' heatScore by Department Over Time'
)

fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig.show()